# Starter notebook
Copy (fork) and edit as many copies of this notebook as you require 

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 36)
pd.set_option("display.max_colwidth", 72)

seed = 42
import numpy as np
np.random.seed(seed)

# graphics
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20})
plt.rcParams["figure.figsize"] = (11, 6.8)
#plt.style.use('fivethirtyeight')

import seaborn as sns
sns.set(font_scale=1)
#sns.set_style("whitegrid")

import plotly.io as pio
# for use in JupyterLab 4
pio.renderers.default = 'iframe'
# for use in Google Colab
#pio.renderers.default = 'colab'
import plotly as py
import plotly.express as px

## Read in the training data

In [ ]:
train = pd.read_csv("/kaggle/input/u-tad-birth-weight-point-prediction-2025/train.csv")
train

## This is the test data that you are asked to make predictions for

In [ ]:
test = pd.read_csv("/kaggle/input/u-tad-birth-weight-point-prediction-2025/test.csv")
test

In [ ]:
#y_pred = regressor.predict(X_test)
y_pred = 1

## Submit your predictions in a `submission.csv` file for scoring on the [leaderboard](https://www.kaggle.com/competitions/u-tad-birth-weight-point-prediction-2025/leaderboard)
To submit your notebook click on **Submit to competition** and then **Submit**.

In [ ]:
# do not modify this code
submission = pd.read_csv("/kaggle/input/u-tad-birth-weight-point-prediction-2025/sample_submission.csv")
submission["DBWT"] = y_pred
submission.to_csv('submission.csv',index=False)

In [ ]:
submission.head()